# Лабораторная 2

Задача: сделать оптимизацию select-запросов
Перед выполнением загрузите файл database.zip и ctrl+F9

Код создания базы данных (для понимания, что в ней находится)

```
CREATE TABLE employees ( -- сотрудники компании
    emp_no      INT             NOT NULL,
    birth_date  DATE            NOT NULL,
    first_name  VARCHAR(14)     NOT NULL,
    last_name   VARCHAR(16)     NOT NULL,
    gender      CHAR(2)         NOT NULL,    
    hire_date   DATE            NOT NULL,
    PRIMARY KEY (emp_no)
);

CREATE TABLE departments ( -- отделения
    dept_no     CHAR(4)         NOT NULL,
    dept_name   VARCHAR(40)     NOT NULL,
    PRIMARY KEY (dept_no)
);

CREATE TABLE dept_manager ( -- менеджеры отделений
   dept_no      CHAR(4)         NOT NULL,
   emp_no       INT             NOT NULL,
   from_date    DATE            NOT NULL,
   to_date      DATE            NOT NULL,
   FOREIGN KEY (emp_no)  REFERENCES employees (emp_no)    ON DELETE CASCADE,
   FOREIGN KEY (dept_no) REFERENCES departments (dept_no) ON DELETE CASCADE,
   PRIMARY KEY (emp_no,dept_no)
); 

CREATE TABLE dept_emp (  -- связь сотрудников и отделений, даты работы
    emp_no      INT             NOT NULL,
    dept_no     CHAR(4)         NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE            NOT NULL,
    FOREIGN KEY (emp_no)  REFERENCES employees   (emp_no)  ON DELETE CASCADE,
    FOREIGN KEY (dept_no) REFERENCES departments (dept_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no,dept_no)
);

CREATE TABLE titles ( -- должности
    emp_no      INT             NOT NULL,
    title       VARCHAR(50)     NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE,
    FOREIGN KEY (emp_no) REFERENCES employees (emp_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no,title, from_date)
); 

CREATE TABLE salaries ( -- зарплаты
    emp_no      INT             NOT NULL,
    salary      INT             NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE            NOT NULL,
    FOREIGN KEY (emp_no) REFERENCES employees (emp_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no, from_date)
); 
```

In [ ]:
import zipfile

with zipfile.ZipFile('database.zip') as zf:
    zf.extractall()

In [ ]:
%load_ext sql
%sql sqlite:///database.db

'Connected: @database.db'

1. Выбрать имена сотрудников с датой рождения до 1 января 1980 года или с должностью помощника инженера 

In [ ]:
%%time
%%sql
SELECT distinct first_name, last_name FROM employees e
INNER JOIN salaries s
    ON s.emp_no = e.emp_no 
    WHERE (
        (e.birth_date < '1980-01-01')
        OR e.emp_no IN (
        select emp_no from titles t
            where t.title like 'Assistant%'
            order by from_date)
    )
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 6.12 ms, sys: 860 µs, total: 6.98 ms
Wall time: 7.44 ms


first_name,last_name
Georgi,Facello
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Kyoichi,Maliniak
Anneke,Preusig
Tzvetan,Zielinski
Saniya,Kalloufi
Sumant,Peac
Duangkaew,Piveteau


In [ ]:
%%time
%%sql
SELECT first_name, last_name FROM employees e
    WHERE birth_date < '1980-01-01'

UNION

SELECT first_name, last_name FROM employees e
    WHERE (
     e.emp_no IN (
        select emp_no from titles t
            where t.title like 'Assistant%')
    )
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 2.83 ms, sys: 0 ns, total: 2.83 ms
Wall time: 2.84 ms


first_name,last_name
Georgi,Facello
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Kyoichi,Maliniak
Anneke,Preusig
Tzvetan,Zielinski
Saniya,Kalloufi
Sumant,Peac
Duangkaew,Piveteau


2. Выбрать сотрудников, в настоящее время работающие в отделах, где менеджеры мужчины

In [ ]:
%%time
%%sql
SELECT first_name, last_name FROM employees e 
INNER JOIN dept_emp d 
    ON d.emp_no = e.emp_no AND d.to_date = (SELECT MAX(to_date) from dept_emp)
    WHERE d.dept_no in (
    SELECT d.dept_no FROM employees e, dept_manager dm, departments d
        WHERE e.emp_no = dm.emp_no AND dm.dept_no = d.dept_no 
        AND dm.to_date = (SELECT MAX(to_date) from dept_manager)
        AND e.gender = 'M'
        GROUP BY d.dept_no
    )
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 36.1 ms, sys: 4.88 ms, total: 40.9 ms
Wall time: 43.2 ms


first_name,last_name
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Sumant,Peac
Duangkaew,Piveteau
Kazuhito,Cappelletti
Cristinel,Bouloucos
Kazuhide,Peha
Mayuko,Warwick
Suzette,Pettey


In [ ]:
%%time
%%sql
SELECT first_name, last_name FROM employees e 
INNER JOIN dept_emp d 
    ON d.emp_no = e.emp_no AND d.to_date = (SELECT MAX(to_date) from dept_emp)
    WHERE d.dept_no in (SELECT distinct d.dept_no FROM departments d
                        INNER JOIN dept_manager dm ON dm.dept_no = d.dept_no AND dm.to_date = (SELECT MAX(to_date) from dept_manager)
                        INNER JOIN employees e ON e.emp_no = dm.emp_no AND e.gender = 'M'
                        )
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 34.2 ms, sys: 3.7 ms, total: 37.9 ms
Wall time: 38.6 ms


first_name,last_name
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Sumant,Peac
Duangkaew,Piveteau
Kazuhito,Cappelletti
Cristinel,Bouloucos
Kazuhide,Peha
Mayuko,Warwick
Suzette,Pettey
